In [1]:
import scanpy as sc
import anndata as ad
import pandas as pd

In [3]:
import os
os.chdir("/dcs04/lieber/lcolladotor/spatialHPC_LIBD4035/spatial_hpc")
os.getcwd()

'/dcs04/lieber/lcolladotor/spatialHPC_LIBD4035/spatial_hpc'

In [4]:
adata = ad.read_h5ad("snRNAseq_hpc/python_analysis/processed-data/adata_raw.h5")
adata.obs['round'] = ["R" + x for x in map(str, adata.obs['round'].map(int))]

In [5]:
postqc = pd.read_csv("snRNAseq_hpc/python_analysis/processed-data/colData_EN-postqc.csv")
postqc.shape

(86905, 18)

In [6]:
key = [x+"-"+y for x, y in zip(postqc['Sample'].values, postqc['Barcode'].values)]
postqc.set_index(pd.Series(key), inplace=True)

In [7]:
adata = adata[postqc.index]
adata

View of AnnData object with n_obs × n_vars = 86905 × 36601
    obs: 'barcode', 'key', 'sample', 'tissue', 'brnum', 'round', 'sorted', 'id', 'dx', 'race', 'sex', 'age', 'pmi'
    var: 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_version', 'gene_type', 'hgnc_id', 'tag', 'havana_gene'

Have to calculate mt fraction so can include in model

In [9]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True, log1p=False)

In [20]:
final = pd.read_csv("snRNAseq_hpc/python_analysis/processed-data/colData_EN-final.csv")
final.shape

(75411, 31)

In [22]:
key = [x+"-"+y for x, y in zip(final['Sample'].values, final['Barcode'].values)]
final.set_index(pd.Series(key), inplace=True)

In [21]:
final.columns
#cell.type2 over celltype

Index(['Sample', 'Barcode', 'key', 'sum', 'detected', 'subsets_Mito_sum',
       'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'high_mito',
       'low_lib', 'low_genes', 'discard_auto', 'discard_semiauto',
       'doubletScore', 'discard', 'brnum', 'round', 'sort', 'sizeFactor',
       'logDoublet', 'k_5_louvain_initial5', 'k_5_louvain_initial',
       'k_5_louvain', 'k_5_louvain2', 'discard2', 'broad.type', 'cell.type',
       'fine.type', 'cell.type2', 'fine.type2'],
      dtype='object')

In [28]:
final.groupby(['fine.type','fine.type2'], observed=True).size()

fine.type    fine.type2 
AHi.1        AHi.1          1481
AHi.2        AHi.2           979
AHi.3        AHi.3          1009
AHi.4        AHi.4            73
Astro.1      Astro.1        1943
Astro.2      Astro.2         274
Astro.3      Astro.3        2081
C1QL1        C1QL1           231
CA1          CA1            3635
CA2          CA2             383
CA3.1        CA3.1          3410
CA3.2        CA3.2           598
COP          COP              57
CORT         CORT            130
CP.1         CP.1           1620
CP.2         CP.2           1218
CP.3         CP.3            328
CRABP1       CRABP1          249
CXCL14       CXCL14          753
Cajal        Cajal            40
Endo.1       Endo.1         1797
Endo.2       Endo.2          183
Ependy       Ependy          421
GC.1         GC.1           4316
GC.2         GC.2           3215
GC.3         GC.3           1260
GC.4         GC.4           1621
GC.5         GC.5             96
HATA         HATA           1982
HTR3A        HTR3A

In [27]:
final.groupby('fine.type2', observed=True).size()

fine.type2
AHi.1          1481
AHi.2           979
AHi.3          1009
AHi.4            73
Astro.1        1943
Astro.2         274
Astro.3        2081
C1QL1           231
CA1            3635
CA2             383
CA3.1          3410
CA3.2           598
COP              57
CORT            130
CP.1           1620
CP.2           1218
CP.3            328
CRABP1          249
CXCL14          753
Cajal            40
Endo.1         1797
Endo.2          183
Ependy          421
GC.1           4316
GC.2           3215
GC.3           1260
GC.4           1621
GC.5             96
HATA           1982
HTR3A           959
L2.3.1         2187
L2.3.2         2113
L2.3.3         1434
L2.3.4          515
L2.3.5          542
L2.3.6          837
L5.1           1811
L5.2            248
L6.1            212
L6.2           1992
L6b            1552
LAMP5.CGE      1208
LAMP5.MGE      1586
MC              928
Macro.Tcell     360
Micro.1        1643
Micro.2        1937
OPC            1605
Oligo.1        3694
Oligo.2  

#### Prep adata

In [11]:
import scvi
import torch
#import rpy2.robjects as ro

In [12]:
rdata = pd.read_csv("snRNAseq_hpc/python_analysis/processed-data/rowData_EN-final_bin-deviance.csv")
hdgs = rdata.gene_name[0:2000]

In [13]:
adata.var['highly_deviant'] = False
adata.var.loc[hdgs,'highly_deviant'] = True

In [14]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

In [15]:
# Subset to highly deviant genes and copy
adata_hdg = adata[:,adata.var["highly_deviant"]].copy()

In [16]:
adata_hdg

AnnData object with n_obs × n_vars = 86905 × 2000
    obs: 'barcode', 'key', 'sample', 'tissue', 'brnum', 'round', 'sorted', 'id', 'dx', 'race', 'sex', 'age', 'pmi', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt'
    var: 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_version', 'gene_type', 'hgnc_id', 'tag', 'havana_gene', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_deviant'
    layers: 'counts'

#### Setting up and running the model

In [17]:
scvi.model.SCVI.setup_anndata(
    adata_hdg,
    layer="counts",
    categorical_covariate_keys=["round", "brnum"],
    continuous_covariate_keys=["pct_counts_mt", "age","pmi"],
)

In [18]:
model = scvi.model.SCVI(adata_hdg)

In [29]:
model.train()

/users/jthompso/.conda/envs/snPy/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /users/jthompso/.conda/envs/snPy/lib/python3.10/site ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/users/jthompso/.conda/envs/snPy/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /users/jthompso/.conda/envs/snPy/lib/python3.10/site ...


Epoch 92/92: 100%|███████████████████████████████████████| 92/92 [33:17<00:00, 22.80s/it, v_num=1, train_loss_step=2.84e+3, train_loss_epoch=3.2e+3]

`Trainer.fit` stopped: `max_epochs=92` reached.


Epoch 92/92: 100%|███████████████████████████████████████| 92/92 [33:17<00:00, 21.71s/it, v_num=1, train_loss_step=2.84e+3, train_loss_epoch=3.2e+3]


In [30]:
model.save("snRNAseq_hpc/python_analysis/processed-data/scvi_model_EN-postqc-2k-hdg", overwrite=True)
#model = scvi.model.SCVI.load(model_dir, adata=adata)